In [1]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
import os
import time
import random
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import phik
from phik.report import plot_correlation_matrix
from phik import report
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

from PM_eq import penman_monteith

In [4]:
mod = np.load('./data_v02/MOD_features_all.npy', allow_pickle=True)
mod_idxs = set(np.load('./data_v02/MOD_target_idx_all.npy', allow_pickle=True))

fluxes = pd.read_csv('./data_v02/target_fluxes_MidWest_LE.csv')
keep_idxs = [idx for idx in fluxes.index.tolist() if (idx in mod_idxs)]
fluxes = fluxes.loc[keep_idxs,:]
fluxes['TIMESTAMP'] = pd.to_datetime(fluxes.TIMESTAMP, format='%Y%m%d')
targets = fluxes.LE_F_MDS.values

fluxes

,TIMESTAMP,LE_F_MDS,LE_F_MDS_QC,NETRAD,G_F_MDS,TA_F,VPD_F,WS_F,PA_F,site,H_F_MDS,lat,lon,IGBP
0,2003-05-24,19.2224,1.0000,236.108854,-1.567770,11.213,10.345,1.841,98.421,FLX_US-Wi1,138.470000,46.7305,-91.2329,DBF
1,2003-05-25,24.2024,1.0000,227.120214,-1.126670,12.308,10.551,1.835,98.471,FLX_US-Wi1,112.405000,46.7305,-91.2329,DBF
2,2003-05-26,19.4017,0.9375,249.112486,0.019932,14.778,12.203,1.660,98.656,FLX_US-Wi1,108.937000,46.7305,-91.2329,DBF
3,2003-05-27,23.7468,1.0000,244.104535,1.914990,18.154,15.802,1.532,98.265,FLX_US-Wi1,112.939000,46.7305,-91.2329,DBF
4,2003-05-28,42.2603,0.9375,246.821474,0.114018,14.890,6.921,2.285,97.722,FLX_US-Wi1,85.043500,46.7305,-91.2329,DBF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155271,2003-09-11,55.7628,1.0000,126.808894,7.353920,23.534,10.864,2.743,96.827,AMF_US-Wi3,15.796000,46.6347,-91.0987,DBF
155272,2003-09-13,31.6040,1.0000,65.703064,-8.120460,15.390,2.544,1.353,96.572,AMF_US-Wi3,-8.821360,46.6347,-91.0987,DBF
155273,2003-09-14,54.2463,1.0000,92.268000,-8.471200,13.966,4.375,1.341,96.912,AMF_US-Wi3,-10.596500,46.6347,-91.0987,DBF
155274,2003-09-15,44.2428,1.0000,99.622415,-3.575530,12.401,3.585,1.304,96.968,AMF_US-Wi3,-0.490375,46.6347,-91.0987,DBF


In [5]:
mod_idxs = pd.Series(list(mod_idxs))
mod_idxs.index = np.arange(len(mod_idxs))
valid_feature_indices = mod_idxs[mod_idxs.isin(fluxes.index)].index.values
mod_features = mod[valid_feature_indices]
mod_features = mod_features[:,:,:-1]#Dropping Coarse_Resolution_Internal_CM

NDVI = (mod_features[:,:,1] - mod_features[:,:,0])/(mod_features[:,:,1] + mod_features[:,:,0])
EVI = (mod_features[:,:,1] - mod_features[:,:,0])/(mod_features[:,:,1] + 6*mod_features[:,:,0] - 7.5*mod_features[:,:,2] + 1)
GNDVI = (mod_features[:,:,1] - mod_features[:,:,3])/(mod_features[:,:,1] + mod_features[:,:,3])
SAVI = (mod_features[:,:,1] - mod_features[:,:,0])/(mod_features[:,:,1] + mod_features[:,:,0] + 0.5)*1.5
ARVI = (mod_features[:,:,1] + mod_features[:,:,2] - 2*mod_features[:,:,0])/(mod_features[:,:,1] + mod_features[:,:,2] + 2*mod_features[:,:,0])

pm_flux = penman_monteith(fluxes, fluxes.index, fluxes, mode='ground').values / 10

features = np.concatenate([mod_features,
                           NDVI[:,:, np.newaxis], EVI[:,:, np.newaxis], GNDVI[:,:, np.newaxis],
                           SAVI[:,:, np.newaxis], ARVI[:,:, np.newaxis]], axis=2)

mod_names = [
          'SensorZenith', 'SensorAzimuth', 'SolarZenith', 'SolarAzimuth',
         'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
         'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 
]

# Model

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the output from the last timestep
        return out

# Training

In [6]:
class CustomDataset(Dataset):
    def __init__(self, X, y, x_scaler=None, y_scaler=None, fit_scalers=False):
        self.x_scaler = x_scaler if x_scaler else MinMaxScaler()
        self.y_scaler = y_scaler if y_scaler else MinMaxScaler()

        if fit_scalers: 
            print('fitting')
            N, T, F = X.shape
            X_reshaped = X.reshape(N, -1)  # Flatten time/feature dims
            self.x_scaler.fit(X_reshaped)
            self.y_scaler.fit(y)

        # Transform data
        N, T, F = X.shape
        X_scaled = self.x_scaler.transform(X.reshape(N, -1)).reshape(N, T, F)
        y_scaled = self.y_scaler.transform(y.reset_index().to_numpy()[:,1].reshape(-1, 1))
        
        self.X = torch.tensor(X_scaled, dtype=torch.float32)
        self.y = torch.tensor(y_scaled, dtype=torch.float32)
        self.target_idx = torch.tensor(y.reset_index().to_numpy()[:,0].reshape(-1, 1), dtype=torch.int32)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.target_idx[idx]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, fluxes, test_size=0.3, random_state=42)

y_train = y_train.LE_F_MDS#reshape(-1, 1)
y_test = y_test.LE_F_MDS#reshape(-1, 1)

x_scaler, y_scaler = MinMaxScaler(), MinMaxScaler()
N, T, F = X_train.shape
x_scaler.fit(X_train.reshape(N, -1))
y_scaler.fit(y_train.reset_index().to_numpy()[:,1].reshape(-1, 1))

train_dataset = CustomDataset(X_train, y_train, x_scaler, y_scaler, fit_scalers=False)
test_dataset = CustomDataset(X_test, y_test, x_scaler, y_scaler, fit_scalers=False)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epoch = 50

input_size = 16 #num features
hidden_size = 128
num_layers = 2
dropout=0.3

In [31]:
model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=1, dropout=dropout)

In [32]:
criteria = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4) 

In [33]:
best = -np.inf
t0 = time.time()
for epoch in tqdm(range(num_epoch)):
    model.train()
    for x, y, idx in train_loader:
        x, y = x.to(torch.float32), y.to(torch.float32)
        pred = model(x)
        error = criteria(pred, y)

        optimizer.zero_grad()
        error.backward()
        optimizer.step()

    if epoch % 5 == 0:
        model.eval()
        test_preds = []
        test_true = []
        test_x = []
        test_idx = []
        with torch.no_grad():
            for x, y, idx in test_loader:
                x, y = x.to(torch.float32), y.to(torch.float32)
                preds = model(x)
                preds = train_dataset.y_scaler.inverse_transform(preds.detach().cpu())
                y_scaled = train_dataset.y_scaler.inverse_transform(y.detach().cpu())
                test_preds.append(torch.tensor(preds))
                test_true.append(torch.tensor(y_scaled))

        test_preds = torch.cat(test_preds).squeeze()
        test_true = torch.cat(test_true).squeeze()
        
        test_loss = criteria(test_preds, test_true).item()
        r2 = r2_score(test_true.numpy(), test_preds.numpy())
        mae = mean_absolute_error(test_true.numpy(), test_preds.numpy())
        #writer.add_scalar("Loss/test", test_loss, epoch)
        #writer.add_scalar("R2/test", r2, epoch)
        
        best_old = best
        best = min(test_loss, best)
        if abs(best) > best_old:
            torch.save(model.state_dict(), f'./models/LSTM.pth') #0.6532
            #writer.add_text("Checkpoint", "Saved model at epoch {}".format(epoch))
        print(f'Test RMSE: {test_loss**0.5:0.3f}\t\tTest R2: {r2:0.4f}\t\t Test MAE: {mae:0.4f}')
#writer.close()

  0%|          | 0/50 [00:00<?, ?it/s]

Test RMSE: 26.450		Test R2: 0.5643		 Test MAE: 19.3334
Test RMSE: 22.483		Test R2: 0.6852		 Test MAE: 15.2189
Test RMSE: 21.106		Test R2: 0.7226		 Test MAE: 14.1092
Test RMSE: 20.852		Test R2: 0.7292		 Test MAE: 14.0054
Test RMSE: 20.630		Test R2: 0.7350		 Test MAE: 13.8653
Test RMSE: 20.687		Test R2: 0.7335		 Test MAE: 14.1849


KeyboardInterrupt: 

In [26]:
r2

0.6740939323066619